### 設定停止點

停止點是用來告訴系統在生成的過程中，除了遇到 EOS (End-of-Sentence) Token 以外，還有遇到哪些 Token 應該停止輸出。

#### 最基本的方法是設定 `GenerationConfig` 的 `eos_token_id`：

In [1]:
from transformers import GenerationConfig
from transformers import TextStreamer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelCls
from transformers import AutoTokenizer as TkCls

model_path = "google/gemma-2b-it"
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkCls.from_pretrained(model_path)

prompt = "[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] "

inputs = tk(prompt, return_tensors="pt").to("cuda")

ts = TextStreamer(tk)

config = GenerationConfig(
    eos_token_id=[
        tk.eos_token_id,       # 留著原本的 EOS
        tk.encode(".")[-1],    # 遇到句點停下
        tk.encode("\n")[-1],   # 遇到換行停下
        tk.encode("\n\n")[-1], # 遇到雙換行停下
    ],
)

output = model.generate(**inputs, generation_config=config, streamer=ts)

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]


<bos>[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] 




#### 自己實作 StoppingCriteria 類別：
* 接著將 `StopWords` 物件放進一個 `StoppingCriteriaList` 裡面，然後傳入 `model.generate` 裡面即可：
    * 每次系統檢查的時候，將整份輸出 Decode 回純文字，並且檢查文字的結尾是否符合使用者設定的停止點。
    * 此範例為單一輸入進行文本生成的情況，批次推論時情況會複雜許多。

In [2]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopWords(StoppingCriteria):
    def __init__(self, tk: TkCls, stop_words: list[str]):
        self.tk = tk
        self.stop_tokens = stop_words

    def __call__(self, input_ids, *_) -> bool:
        s = self.tk.batch_decode(input_ids)[0]
        for t in self.stop_tokens:
            if s.endswith(t):
                return True
        return False


sw = StopWords(tk, ["。", "！", "？"])
scl = StoppingCriteriaList([sw])
print(scl)

output = model.generate(
    **inputs,
    max_new_tokens=2048,
    streamer=TextStreamer(tk),
    stopping_criteria=scl,
)
# 模型在輸出遇到 "。！？" 時就會停下來了。

<bos>[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] 

大型語言模型（LLM）是一種 AI 模型，它能夠像人類一樣使用語言。


#### Transformers 在 `GenerationConfig` 中新增了 `stop_strings` 的參數：
* 可以更輕鬆的設定停止點：

In [9]:
from transformers import GenerationConfig

config = GenerationConfig(
    stop_strings=[".", "\n"],
)

inputs = tk(["hello,", "goodbye, "], padding=True, return_tensors="pt")
inputs = inputs.to(model.device)
output = model.generate(**inputs, generation_config=config, tokenizer=tk)

print(tk.decode(output[0], skip_special_tokens=True))


hello, i'm looking for a way to make my website more accessible to people with disabilities.
